In [1]:
# pip install langchain --upgrade
# Version: 0.0.164

! pip install pypdf


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [27]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load your data

In [26]:
loader = PyPDFLoader("../data/UHV Text Book.pdf")

## Other options for loaders 
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [29]:
data = loader.load()

In [ ]:
data

In [31]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 314 document(s) in your data
There are 137 characters in your document


### Chunk your data up into smaller documents

In [ ]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
texts = text_splitter.split_documents(data)
texts

In [54]:
print (f'Now you have {len(texts)} documents')

Now you have 488 documents


### Create embeddings of your documents to get ready for semantic search

In [38]:
# Why is it used
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os

In [39]:
# Check to see if there is an environment variable with you API keys, if not, use what you put below
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', "us-west1-gcp-free") # You may need to switch with your env

In [40]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [41]:
def create_index(index_name):
    pinecone.create_index(index_name, dimension=1536,  metric='cosine')

def delete_index(index_name):
    pinecone.delete_index(index_name)

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "uhv-book-index" # put in the name of your pinecone index here
create_index(index_name)

### Helper Functions

In [43]:
# No need to load any pdf, use existing index to query.
def get_docs_from_exsting_index():
    return Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

# Need to load pdf first.
def get_docs_from_text():
    return Pinecone.from_texts(texts=[doc.page_content for doc in texts], metadatas=[doc.metadata for doc in texts], embedding=embeddings, index_name=index_name)

In [44]:
docsearch = get_docs_from_text()

In [ ]:
# docsearch = Pinecone.from_existing_index(index_name="langchain1", embedding=embeddings)

In [45]:
docsearch

### Query those docs to get your answer back

In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [103]:
prompt_template = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Answer the question in 50-90 words. Just write answer don't use "based on given context".

{context}

Question: {question}
Answer:
"""
prompt = PromptTemplate(
    template = prompt_template,
    input_variables=["context", "question"]
)
#Load an LLM
# llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",            
)

#Setup chain
chain = load_qa_chain(
    llm=llm,
    chain_type="stuff",
    prompt = prompt # Add custom prompt
)


In [104]:
chain.llm_chain.prompt

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='\nUse the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer. Answer the question in 50-90 words. Just write answer don\'t use "based on given context".\n\n{context}\n\nQuestion: {question}\nAnswer:\n', template_format='f-string', validate_template=True)

In [112]:
loader = PyPDFLoader("../data/UHV Question Bank_single_line.pdf")
docs = loader.load()
pages ="".join(doc.page_content for doc in docs)
queries = [query.strip() for query in pages.split("\n") if query.strip()]
queries

['1  1. Explain the feelings of respect,  glory, reverence, and gratitude.',
 '2. How can we practice reverence daily?',
 '3. Does reverence mean respect?',
 '4. How do people show reverence?',
 '5. Explain the concept of false reverence.',
 '6. What are the steps required to fulfill our basic aspiration?',
 '7. Define Aspiration. State the link between effort and aspiration.',
 '8. Why is there a gap between our present effort and expectation to get something?',
 '9. Diagrammatically represent life with clarity of  basic aspiration.',
 '10. Define effort.',
 "11. How does value education helps in fulfilling one's aspirations?",
 '12. What is the meaning of prosperity? How can you say that you are prosperous?',
 '13. What is the program to fulfil the basic human aspirations? Explain',
 '14. Do you th ink that human beings are sum -total of sentiments and physical aspects the ‘self’ and the ‘body’? Explain your answer using examples.',
 '15. What do you understand  by fulfilment  of bas

In [101]:
import json
def save_answer(answers: dict, file_name: str):
    with open(f"../data/{file_name}", "w", encoding="utf-8") as fp:
        json.dump(answers, fp, indent=2, ensure_ascii=True)

def read_answers(file_name: str) -> dict:
    with open(f"../data/{file_name}", "r") as fp:
        answers = json.load(fp)
        return answers

In [127]:
answers = read_answers(file_name="UHV_Answers_gpt-3.5-turbo.json")
lines= []
for question, answer_details in answers.items():
    answer = answer_details.get("answer")
    pages = answer_details.get("pages")
    lines.append(f"### {question}\n")
    lines.append(f"{answer}\n")
    lines.append(f" - Source: {pages}\n\n\n")
with open("../data/answers.md", "w") as f:
    f.writelines(lines)
# save_answer(answers, file_name="UHV_Answers_gpt-3.5-turbo.json")

In [51]:
pinecone.describe_index(index_name)

IndexDescription(name='uhv-book-index', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [115]:
from collections import defaultdict
file_name = "UHV_Answers_gpt-3.5-turbo.json"
answers_dict = read_answers(file_name=file_name)
sub_queries = queries[53:]
for query in sub_queries:
    docs = docsearch.similarity_search(query)
    details = defaultdict()
    answer = chain.run(input_documents=docs, question=query)
    print(f"{query = }\n{answer = }\n")
    details["answer"] = answer
    details["pages"] = [int(doc.metadata.get("page"))+1 for doc in docs]
    answers_dict[query] = details
    save_answer(answers_dict, file_name = file_name)

query = '55. When does mutual growth /competence gets better while we work in collaboration or in isolation. Illustrate with relevant exam ples.'
answer = 'Mutual growth and competence are better achieved when working in collaboration rather than in isolation. When individuals collaborate, they can share their knowledge, skills, and perspectives, leading to a more comprehensive understanding and better outcomes. For example, in a team project, each member can contribute their unique expertise, resulting in a more innovative and successful solution. On the other hand, working in isolation limits the exchange of ideas and hinders the development of new insights and skills. Therefore, collaboration fosters mutual growth and competence by leveraging the collective strengths and abilities of individuals.'

query = '56. What is the proposal for the definition of trust? Illustrate t he feeling of trust with one'
answer = "The proposal for the definition of trust is to differentiate between in